# How happy are Zürcher with Tempo 30?

Dataset from [Daten der Bevölkerungsbefragung der Stadt Zürich 1999 bis 2019](https://data.stadt-zuerich.ch/dataset/prd_stez_bevoelkerungsbefragungen_seit1999_od4730)

We want to compare this with the kilometers (or number of roads) with Tempo 30 in each Kreis :
- [Verkehrszonen](https://www.ogd.stadt-zuerich.ch/geodaten/Verkehrszonen?format=10009)

---

You need to unzip `bev473od4731.zip` before running this notebook.

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('bev473od4731.csv')
df.shape

(28370, 318)

In [4]:
df.head()

,StichtagDatJahr,intnr,methode,fbversion,sgewicht,pgewicht,kreis,s1,f1,f2,...,f58_8,f58_9,f58_10,f58_11,f58_12,f58_13,f59,f60a,f60b,f60c
0,2019,1101,online,Unbekannt,0.279589,17.750000,Kreis 01/02,Frau,Eher gerne,Note 4,...,Ja,Nein,Nein,Nein,Nein,Nein,Sehr zufrieden (6),Sehr zufrieden (6),Sehr zufrieden (6),Sehr zufrieden (6)
1,2019,1102,online,Unbekannt,1.020983,64.818182,Kreis 07,Frau,Eher gerne,Note 5,...,Nein,Nein,Nein,Nein,Nein,Nein,3,4,3,3
2,2019,1103,online,Unbekannt,0.551302,35.000000,Kreis 01/02,Frau,Sehr gerne,Note 6 (sehr gut),...,Ja,Nein,Nein,Nein,Nein,Nein,Sehr zufrieden (6),4,5,5
3,2019,1104,online,Unbekannt,0.989136,62.796296,Kreis 09,Frau,Sehr ungern,Note 6 (sehr gut),...,Nein,Nein,Nein,Nein,Nein,Nein,4,5,5,5
4,2019,1105,online,Unbekannt,0.986286,62.615385,Kreis 01/02,Frau,Sehr gerne,Note 5,...,Nein,Nein,Nein,Nein,Ja,Nein,3,3,3,4


- [Renaming columns](https://stackoverflow.com/a/46096346/5239250)

In [5]:
happy = df[['kreis', 'f1', 'f4l']].rename(columns={'f1': 'happy_zh', 'f4l': 'happy_30'})

happy.head()

,kreis,happy_zh,happy_30
0,Kreis 01/02,Eher gerne,Weiss nicht
1,Kreis 07,Eher gerne,Weiss nicht
2,Kreis 01/02,Sehr gerne,Note 4
3,Kreis 09,Sehr ungern,Note 6 (sehr zufrieden)
4,Kreis 01/02,Sehr gerne,Note 5


https://chrisalbon.com/python/data_wrangling/pandas_list_unique_values_in_column/

In [6]:
happy.happy_zh.unique()

array(['Eher gerne', 'Sehr gerne', 'Sehr ungern', 'Eher ungern',
       'Keine Angabe'], dtype=object)

In [7]:
happy = happy[happy.happy_zh != 'Keine Angabe']
categories = {'happy_zh': {'Sehr gerne': 4, 'Eher gerne': 3, 'Eher ungern': 2, "Sehr ungern": 1}}
happy.replace(categories, inplace=True)
happy.head()

,kreis,happy_zh,happy_30
0,Kreis 01/02,3,Weiss nicht
1,Kreis 07,3,Weiss nicht
2,Kreis 01/02,4,Note 4
3,Kreis 09,1,Note 6 (sehr zufrieden)
4,Kreis 01/02,4,Note 5


- [How do I remove unwanted parts from strings in a column?](https://stackoverflow.com/a/54302517/5239250)

Removing all but the digits leaves us with a number as string, which we have to convert to numeric.

In [8]:
happy.happy_30.unique()

array(['Weiss nicht', 'Note 4', 'Note 6 (sehr zufrieden)', 'Note 5',
       'Note 2', 'Note 3', 'Note 1 (sehr unzufrieden)', 'Keine Angabe'],
      dtype=object)

In [9]:
happy = happy[happy.happy_30 != "Weiss nicht"]
happy = happy[happy.happy_30 != "Keine Angaben"]
happy.happy_30 = happy.happy_30.str.extract('(\d+)', expand=False)
happy.happy_30 = pd.to_numeric(happy.happy_30)
happy.head()


,kreis,happy_zh,happy_30
2,Kreis 01/02,4,4.0
3,Kreis 09,1,6.0
4,Kreis 01/02,4,5.0
6,Kreis 10,4,4.0
7,Kreis 11,4,2.0


In [12]:
happy.groupby('kreis').mean()

,happy_zh,happy_30
kreis,,
Kreis 01/02,3.757342,4.309773
Kreis 03,3.751210,4.470559
Kreis 04,3.702027,4.521555
Kreis 05,3.753425,4.518968
Kreis 06,3.783482,4.506131
Kreis 07,3.778970,4.325750
Kreis 08,3.803717,4.248266
Kreis 09,3.708949,4.296989
Kreis 10,3.777013,4.337150


happy_zh: 1-4; happy_30: 1-6